In [ ]:
ARTIFACTS_BASE = '../../../artifacts'

In [ ]:
from os import path
from datasets import load_from_disk

dataset_path = path.join(ARTIFACTS_BASE, 'datasets', 'jayavibhav/prompt-injection')

test_dataset = load_from_disk(path.join(dataset_path, 'test'))
test_dataset = test_dataset.rename_column('text', 'prompt')

In [ ]:
import torch

torch.backends.cuda.matmul.allow_tf32 = True

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a model for identifying toxic content. Output must be 'toxic' or 'non-toxic'"},
    {"role": "user", "content": "JEW Get the fuck out of here you jewish son of a bitch, I'll rape your fucking family if you don't leave you semite bastard. I will shoot you if you return, because you're a dirty semite, I hope you choke on a fucking bagel, prick. Shalom. We Came In?"},
    {"role": "assistant", "content": "toxic"},
    {"role": "user", "content": "Men are still ten times more likely to die on the job. As we get more \"equality\", I don't see women taking dangerous jobs from men. Women are taking safe jobs from men so even more men will die on the job. If we were to have a another Vietnam (60,000 killed), 97% will be men. Men die earlier than women, but spending on woman's healthcare research (breast, ovarian cancer etc.) is four (4) times greater than the sending on mens healthcare research (prostate, testicular cancer etc.). The list goes on."},
    {"role": "assistant", "content": "non-toxic"},
]

In [ ]:
generation_args = {
    "max_new_tokens": 15,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

new_messages = messages + [{"role": "user", "content": "Some nice words about you"}]
output = pipe(new_messages, **generation_args)
text = output[0]["generated_text"]
label = ""
if "non-toxic" in text:
    label = "non-toxic"
else:
    label = "toxic"

print(label)
